In [2]:
:clear_cache

Deleted 0 MiB from cache


In [3]:
:dep arrow = { version = "54.2.1"}
:dep rand = {version = "0.9.0"}
:dep rand_distr = {version = "0.5.1"}

In [8]:
use std::sync::Arc;
use arrow::{
    array::{Float64Builder, Int16Builder, Int64BufferBuilder, StringBuilder, StringArray, StructArray},
    buffer::Buffer,
    datatypes::{DataType, Field, Schema},
    error::Result,
    record_batch::RecordBatch
};
use rand;
use rand_distr::Normal;

In [9]:
:help

:allow_static_linking,Set whether to allow static linking of dependencies (0/1)
:build_env,Set environment variables when building code (key=value)
:cache,"Set cache size in MiB, or 0 to disable."
:clear,"Clear all state, keeping compilation cache"
:clear_cache,Clear the cache used by the :cache command
:codegen_backend,Set/print the codegen backend. Requires nightly
:dep,"Add dependency. e.g. :dep regex = ""1.0"""
:doc,"show the documentation of a variable, keyword, type or module"
:efmt,Set the formatter for errors returned by ?
:env,Set an environment variable (key=value)
:explain,Print explanation of last error


In [21]:
:doc arrow

extern crate arrow


A complete, safe, native Rust implementation of Apache Arrow, a cross-language development platform for in-memory data.

Please see the arrow crates.io page for feature flags and tips to improve performance.

Columnar FormatThe [array] module provides statically typed implementations of all the array types as defined by the Arrow Columnar Format

For example, an Int32Array represents a nullable array of i32

# use arrow::array::{Array, Int32Array};
let array = Int32Array::from(vec![Some(1), None, Some(3)]);
assert_eq!(array.len(), 3);
assert_eq!(array.value(0), 1);
assert_eq!(array.is_null(1), true);

let collected: Vec<_> = array.iter().collect();
assert_eq!(collected, vec![Some(1), None, Some(3)]);
assert_eq!(array.values(), &[1, 0, 3])

It is also possible to write generic code for different concrete types. For example, since the following function is generic over all primitively typed arrays, when invoked the Rust compiler will generate specialized implementations with optimized code for each concrete type.

# use std::iter::Sum;
# use arrow::array::{Float32Array, PrimitiveArray, TimestampNanosecondArray};
# use arrow::datatypes::ArrowPrimitiveType;
#
fn sum<T: ArrowPrimitiveType>(array: &PrimitiveArray<T>) -> T::Native
where
    T: ArrowPrimitiveType,
    T::Native: Sum
{
    array.iter().map(|v| v.unwrap_or_default()).sum()
}

assert_eq!(sum(&Float32Array::from(vec![1.1, 2.9, 3.])), 7.);
assert_eq!(sum(&TimestampNanosecondArray::from(vec![1, 2, 3])), 6);

And the following uses [ArrayAccessor] to implement a generic function over all arrays with comparable values.

# use arrow::array::{ArrayAccessor, ArrayIter, Int32Array, StringArray};
# use arrow::datatypes::ArrowPrimitiveType;
#
fn min<T: ArrayAccessor>(array: T) -> Option<T::Item>
where
    T::Item: Ord
{
    ArrayIter::new(array).filter_map(|v| v).min()
}

assert_eq!(min(&Int32Array::from(vec![4, 2, 1, 6])), Some(1));
assert_eq!(min(&StringArray::from(vec!["b", "a", "c"])), Some("a"));

For more examples, and details consult the arrow_array docs.

Type Erasure / Trait ObjectsIt is common to write code that handles any type of array, without necessarily knowing its concrete type. This is done using the [Array] trait and using [DataType] to determine the appropriate downcast_ref.

# use arrow::array::{Array, Float32Array};
# use arrow::array::StringArray;
# use arrow::datatypes::DataType;
#
fn impl_string(array: &StringArray) {}
fn impl_f32(array: &Float32Array) {}

fn impl_dyn(array: &dyn Array) {
    match array.data_type() {
        // downcast `dyn Array` to concrete `StringArray`
        DataType::Utf8 => impl_string(array.as_any().downcast_ref().unwrap()),
        // downcast `dyn Array` to concrete `Float32Array`
        DataType::Float32 => impl_f32(array.as_any().downcast_ref().unwrap()),
        _ => unimplemented!()
    }
}

You can use the [AsArray] extension trait to facilitate downcasting:

# use arrow::array::{Array, Float32Array, AsArray};
# use arrow::array::StringArray;
# use arrow::datatypes::DataType;
#
fn impl_string(array: &StringArray) {}
fn impl_f32(array: &Float32Array) {}

fn impl_dyn(array: &dyn Array) {
    match array.data_type() {
        DataType::Utf8 => impl_string(array.as_string()),
        DataType::Float32 => impl_f32(array.as_primitive()),
        _ => unimplemented!()
    }
}

It is also common to want to write a function that returns one of a number of possible array implementations. [ArrayRef] is a type-alias for Arc<dyn Array> which is frequently used for this purpose

# use std::str::FromStr;
# use std::sync::Arc;
# use arrow::array::{ArrayRef, Int32Array, PrimitiveArray};
# use arrow::datatypes::{ArrowPrimitiveType, DataType, Int32Type, UInt32Type};
# use arrow::compute::cast;
#
fn parse_to_primitive<'a, T, I>(iter: I) -> PrimitiveArray<T>
where
    T: ArrowPrimitiveType,
    T::Native: FromStr,
    I: IntoIterator<Item=&'a str>,
{
    PrimitiveArray::from_iter(iter.into_iter().map(|val| T::Native::from_

In [17]:
fn first_example() {
    let data = [1,2,3,4];
    let mut builder = Int64BufferBuilder::new(data.len());
    builder.append_slice(&data);
    let arr = builder.finish();
    println!("{:?}", arr);
}

first_example()

Buffer { data: Bytes { ptr: 0x14af04100, len: 32, data: [1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0] }, ptr: 0x14af04100, length: 32 }


()

In [20]:
let archers = StringArray::from(
    vec!["Legolas", "Oliver", "Merida", "Lara", "Artemis"]
);
archers

StringArray
[
  "Legolas",
  "Oliver",
  "Merida",
  "Lara",
  "Artemis",
]